In [ ]:
import numpy as np
import torch

from s3ts.helper_functions import *

In [ ]:
dm = load_tsdataset("WISDM", "./datasets/", 128, 8, 48, 1, True, None, [30, 31, 32, 33, 34, 35], True, 1)

In [ ]:
from s3ts.api.nets.methods import create_model_from_DM, train_model

model = create_model_from_DM(dm, "ts", "cnn", "mlp", "cls", "test", 20, 32, 2, 0.001)

In [ ]:
train_model(dm, model, 5, {
    "default_root_dir": "training",
    "accelerator": "gpu",
    "seed": 42
})

In [ ]:
import torchmetrics as tm

In [ ]:
auroc_online = tm.AUROC(task="multiclass", num_classes=6, average="macro")
dataloader = dm.test_dataloader()

In [ ]:
auroc_batch = []

In [ ]:
auroc_online.reset()
for batch_idx, data in enumerate(dataloader):
    preds = model.logits(data["series"])
    auroc_batch.append(auroc_online(preds, data["label"]))

In [ ]:
auroc_online.compute()